In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
pd.options.mode.chained_assignment = None

In [2]:
from sklearn.metrics import confusion_matrix,average_precision_score,auc,roc_curve,roc_auc_score,precision_recall_curve

In [3]:
df=pd.read_csv('credit_card.csv')
#df=data.copy()
df.shape

(9856, 33)

In [3]:
df=pd.read_csv('credit_card.csv')
#df = dat.sample(n=8800)
data=df.copy()
data.shape

(9856, 33)

In [4]:
data.columns

Index(['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8',
       'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',
       'Amount', 'Class', 'Outlier'],
      dtype='object')

In [6]:
N = df.shape[0]      # No of records in data
k = np.int(np.ceil(N*0.12))  # No of outliers to be inserted
k_rand_ind = random.sample(range(0,N),k)
k_rand_pt = df.iloc[k_rand_ind,:]
k_rand_pt1 = k_rand_pt.copy()
z_rand_list = []
y_features = ['Amount']

for i in range(k):
    # Randomely choose a record from data
    z_rand = pd.DataFrame()
    z_rand = k_rand_pt.iloc[i:i+1,:]

    # Use the algorithm to exhange y_i with y_j
    p = np.int(np.ceil(min(50, N/4)))
    p_rand_ind = random.sample(range(0,N),p)
    p_rand_pt = df.iloc[p_rand_ind,:]
    for col in y_features:
        p_rand_pt['d'] = np.sqrt((p_rand_pt[col]-z_rand[col].values[0])**2)
        z_max_d = p_rand_pt[p_rand_pt.d==max(p_rand_pt.d)].head(1)
        z_rand.loc[:, col]=z_max_d[col].values

    z_rand_list.append(z_rand)
    
z_new = pd.concat(z_rand_list, ignore_index=True)  # Final outliers data
data['Train']=0
z_new['Train']=1
final_df = pd.concat([data, z_new], ignore_index=True)   # final data with outliers inserted
final_df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Train
9851,141921.0,-0.655473,0.909024,0.076747,-1.091129,1.498395,-0.499196,1.791069,-1.122708,1.240518,...,-0.638282,-0.015950,-1.000150,-0.245207,0.085624,-0.332070,-0.564749,1080.06,0,1
9852,69641.0,-0.756761,0.169398,1.121188,-1.575503,-0.798940,-0.927480,-0.244460,0.472491,-1.387783,...,0.398377,-0.020969,0.515605,-0.323048,-0.509758,-0.004371,0.048925,1196.88,0,1
9853,159361.0,0.055017,0.845602,0.227417,-0.617479,0.457709,-1.048229,0.990579,-0.183883,0.003482,...,-0.618804,0.053347,-0.124007,-0.478136,0.148284,0.245518,0.095360,1245.58,0,1
9854,35776.0,0.832233,-1.214110,-0.193996,-0.689449,-0.591140,0.038566,-0.086848,0.034861,-1.358453,...,-0.515731,-0.035270,-0.308233,0.077768,-0.513227,-0.033040,0.039211,0.76,0,1
9855,137877.0,1.886377,-2.532818,-1.258221,-2.431110,0.062551,4.224351,-2.486958,1.125434,-0.610150,...,0.554542,0.167994,0.703871,-0.553807,-0.088264,0.069885,-0.003408,1150.00,0,1


In [54]:
final_df.to_csv('cod_credit_card.csv')

In [7]:
final_df.query('Train==1').shape

(1056, 32)

In [5]:
df.columns

Index(['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8',
       'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18',
       'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',
       'Amount', 'Class', 'Outlier'],
      dtype='object')

In [4]:
x_test=df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']]
y_test=df[['Amount']]
test_o=df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28','Amount']]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scale=scaler.fit(x_test)
x_scaled=scaler.transform(x_test)

In [6]:
scaled_dat=scaler.fit(test_o)
scaled_data=scaler.transform(test_o)

In [11]:
x_test.shape

(9856, 29)

In [9]:
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras.optimizers import RMSprop 
from keras.callbacks import EarlyStopping 


#define base model
model = Sequential()
model.add(Dense(128, kernel_initializer='normal',input_dim = 29, activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               3840      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 102,913
Trainable params: 102,913
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(
   x_scaled, 
   y_test,    
   batch_size=128, 
   epochs = 500, 
   verbose = 1, 
   validation_split = 0.2, 
   callbacks = [EarlyStopping(monitor = 'val_loss', patience = 30)]
)

Epoch 1/500
62/62 [==============================] - 2s 20ms/step - loss: 83.8856 - mean_absolute_error: 83.8856 - val_loss: 574.1733 - val_mean_absolute_error: 574.1733
Epoch 2/500
62/62 [==============================] - 0s 7ms/step - loss: 52.9484 - mean_absolute_error: 52.9484 - val_loss: 566.5778 - val_mean_absolute_error: 566.5778
Epoch 3/500
62/62 [==============================] - 0s 7ms/step - loss: 18.1135 - mean_absolute_error: 18.1135 - val_loss: 561.1825 - val_mean_absolute_error: 561.1825
Epoch 4/500
62/62 [==============================] - 0s 8ms/step - loss: 13.0826 - mean_absolute_error: 13.0826 - val_loss: 559.3410 - val_mean_absolute_error: 559.3410
Epoch 5/500
62/62 [==============================] - 0s 8ms/step - loss: 10.7246 - mean_absolute_error: 10.7246 - val_loss: 559.7989 - val_mean_absolute_error: 559.7989
Epoch 6/500
62/62 [==============================] - 0s 6ms/step - loss: 9.5575 - mean_absolute_error: 9.5575 - val_loss: 558.0272 - val_mean_absolute_err

Epoch 50/500
62/62 [==============================] - 0s 7ms/step - loss: 3.0715 - mean_absolute_error: 3.0715 - val_loss: 555.8090 - val_mean_absolute_error: 555.8090
Epoch 51/500
62/62 [==============================] - 0s 7ms/step - loss: 3.3241 - mean_absolute_error: 3.3241 - val_loss: 556.8311 - val_mean_absolute_error: 556.8311
Epoch 52/500
62/62 [==============================] - 0s 7ms/step - loss: 3.3554 - mean_absolute_error: 3.3554 - val_loss: 556.2033 - val_mean_absolute_error: 556.2033
Epoch 53/500
62/62 [==============================] - 0s 8ms/step - loss: 3.0481 - mean_absolute_error: 3.0481 - val_loss: 556.7836 - val_mean_absolute_error: 556.7836
Epoch 54/500
62/62 [==============================] - 0s 8ms/step - loss: 3.0815 - mean_absolute_error: 3.0815 - val_loss: 555.7676 - val_mean_absolute_error: 555.7676
Epoch 55/500
62/62 [==============================] - 0s 8ms/step - loss: 2.7777 - mean_absolute_error: 2.7777 - val_loss: 556.0664 - val_mean_absolute_error: 5

Epoch 99/500
62/62 [==============================] - 0s 7ms/step - loss: 2.2089 - mean_absolute_error: 2.2089 - val_loss: 555.4316 - val_mean_absolute_error: 555.4316
Epoch 100/500
62/62 [==============================] - 0s 7ms/step - loss: 2.1090 - mean_absolute_error: 2.1090 - val_loss: 555.4927 - val_mean_absolute_error: 555.4927
Epoch 101/500
62/62 [==============================] - 0s 7ms/step - loss: 2.5620 - mean_absolute_error: 2.5620 - val_loss: 555.1531 - val_mean_absolute_error: 555.1531
Epoch 102/500
62/62 [==============================] - 0s 8ms/step - loss: 2.4389 - mean_absolute_error: 2.4389 - val_loss: 555.9759 - val_mean_absolute_error: 555.9759
Epoch 103/500
62/62 [==============================] - 0s 7ms/step - loss: 1.9951 - mean_absolute_error: 1.9951 - val_loss: 555.5436 - val_mean_absolute_error: 555.5436
Epoch 104/500
62/62 [==============================] - 0s 8ms/step - loss: 1.9668 - mean_absolute_error: 1.9668 - val_loss: 555.6574 - val_mean_absolute_err

In [11]:
from sklearn.metrics import r2_score, mean_squared_error
mse=mean_squared_error(y_test, model.predict(x_scaled))
r2 =r2_score(y_test, model.predict(x_scaled))
print('R2_score:',r2)
print('Mean_squared_error:',mse)

R2_score: 0.09954198014037596
Mean_squared_error: 264249.96619479026


In [12]:
test_outcome=df.copy()
y_pred_test=np.round_(model.predict(x_scaled),decimals=4)
test_outcome['test_prediction']=y_pred_test
diff=np.absolute(test_outcome['test_prediction']-test_outcome['Amount'])
#norm=np.sqrt(difference)
#weight=np.max(r2,0)
#weighted_score=np.round(norm*weight,decimals=2)
test_outcome['out_score']=diff
test_result=test_outcome[['Amount','test_prediction','out_score']]
test_result.tail()

,Amount,test_prediction,out_score
9851,1080.06,10.189900,1069.870100
9852,1196.88,15.535300,1181.344700
9853,1245.58,1.980600,1243.599400
9854,0.76,252.800003,252.040003
9855,1150.00,178.357803,971.642197


In [36]:
from pyod.models.cof import COF
clf=COF()
clf.fit(test_result[['out_score']])

COF(contamination=0.1, method='fast', n_neighbors=None)

In [37]:
y_pred_cod=clf.labels_
y_prob_cod=clf.predict_proba(test_result[['out_score']])[:,1]
y_true=df[['Outlier']]
confusion_matrix(y_true,y_pred_cod)

array([[7998,  802],
       [ 872,  184]], dtype=int64)

In [38]:
#Isolation Forest results
fpr,tpr,threshold=roc_curve(y_true,y_prob_cod,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_cod))
print('auc: %.4f' % auc(fpr,tpr))

Average prcision: 0.1210
auc: 0.5380


In [16]:
##KNN based COD detection
from pyod.models.knn import KNN
kn=KNN(metric='minkowski',n_neighbors=20)
kn.fit(scaled_data)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=20, p=2,
  radius=1.0)

In [17]:
##KNN based COD detection
y_pred_kn=kn.labels_
y_prob_kn=kn.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_kn)

array([[7964,  836],
       [ 906,  150]], dtype=int64)

In [18]:
##KNN based COD detection result
fpr_kn,tpr_kn,threshold_kn=roc_curve(y_true,y_prob_kn,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_kn))
print('auc: %.4f' % auc(fpr_kn,tpr_kn))

Average prcision: 0.1135
auc: 0.6144


In [19]:
#LOF analysis
from pyod.models.lof import LOF
clf_lof=LOF(n_neighbors=20)
clf_lof.fit(scaled_data)

LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=False, p=2)

In [20]:
##KNN based COD detection
y_pred_lf=clf_lof.labels_
y_prob_lf=clf_lof.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_lf)

array([[7979,  821],
       [ 891,  165]], dtype=int64)

In [21]:
#LOF results
fpr_lf, tpr_lf, thresholds_lf = roc_curve(y_true, y_prob_lf,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_lf))
print('AUC_score: %.4f' % auc(fpr_lf, tpr_lf))

Average prcision: 0.1165
AUC_score: 0.6464


In [10]:
#COF analysis
from pyod.models.ecod import ECOD
clf_cof=ECOD()
clf_cof.fit(scaled_data)

ModuleNotFoundError: No module named 'pyod.models.ecod'

In [23]:
##COF based COD detection
y_pred_cf=clf_cof.labels_
y_prob_cf=clf_cof.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_cf)

array([[7959,  841],
       [ 911,  145]], dtype=int64)

In [24]:
#COF results
fpr_cf, tpr_cf, thresholds_cf = roc_curve(y_true, y_prob_cf,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_cf))
print('AUC_score: %.4f' % auc(fpr_cf, tpr_cf))

Average prcision: 0.1126
AUC_score: 0.5817


In [25]:
#IForest analysis
from pyod.models.iforest import IForest
clf_if=IForest()
clf_if.fit(scaled_data)

IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0)

In [26]:
##IForest based COD detection
y_pred_if=clf_if.labels_
y_prob_if=clf_if.predict_proba(scaled_data)[:,1]
confusion_matrix(y_true,y_pred_if)

array([[7955,  845],
       [ 915,  141]], dtype=int64)

In [27]:
#IForest results
fpr_if, tpr_if, thresholds_if = roc_curve(y_true, y_prob_if,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_if))
print('AUC_score: %.4f' % auc(fpr_if, tpr_if))

Average prcision: 0.1119
AUC_score: 0.6254


In [28]:
test_o.shape

(9856, 30)

In [7]:
#COPOD analysis
from pyod.models.sos import SOS
clf_va=SOS()
clf_va.fit(scaled_data)

SOS(contamination=0.1, eps=1e-05, metric='euclidean', perplexity=4.5)

In [8]:
##COPOD based COD detection
y_pred_va=clf_va.labels_
y_prob_va=clf_va.predict_proba(scaled_data)[:,1]
y_true=df[['Outlier']]
confusion_matrix(y_true,y_pred_va)

array([[7893,  907],
       [ 977,   79]], dtype=int64)

In [9]:
#COPOD results
fpr_va, tpr_va, thresholds_va = roc_curve(y_true, y_prob_va,pos_label=1)
print('Average prcision: %.4f' % average_precision_score(y_true, y_pred_va))
print('AUC_score: %.4f' % auc(fpr_va, tpr_va))

Average prcision: 0.1051
AUC_score: 0.4951


In [43]:
#ROC analysis
import plotly.graph_objects as go

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1)

fig.add_trace(go.Scatter(x=fpr, y=tpr, name=f'HINCOD(AUC={auc(fpr, tpr):.3f})', mode='lines',marker_color='crimson'))
fig.add_trace(go.Scatter(x=fpr_kn, y=tpr_kn, name=f'KNN(AUC={auc(fpr_kn, tpr_kn):.3f})', mode='lines',marker_color='yellow'))
fig.add_trace(go.Scatter(x=fpr_lf, y=tpr_lf, name=f'LOF(AUC={auc(fpr_lf, tpr_lf):.3f})', mode='lines'))
fig.add_trace(go.Scatter(x=fpr_cf, y=tpr_cf, name=f'COF(AUC={auc(fpr_cf, tpr_cf):.3f})', mode='lines'))
fig.add_trace(go.Scatter(x=fpr_if, y=tpr_if, name=f'IForest(AUC={auc(fpr_if, tpr_if):.3f})', mode='lines',marker_color='hotpink'))



fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=600)
fig.show()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(data)

In [ ]:
ts=pd.DataFrame(projections)
ts.columns=['tsne_feature1','tsne_feature2']

In [ ]:
y_out=df['Out'].astype('category')

In [ ]:
ts['outlier']=y_out

In [ ]:
import plotly.express as px
fig = px.scatter(ts, x='tsne_feature1', y='tsne_feature2',color='outlier')
fig.show()